In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from lib.vllm import start_vllm_server, vllm_server_metrics

model = "NousResearch/Hermes-2-Theta-Llama-3-8B"

shutdown_server, client = await start_vllm_server(
    disable_log_requests=True,
    model=model,
)

/home/ubuntu/atreides/.venv/lib/python3.12/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.54it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.29it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.68it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.55it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.53it/s]



In [26]:
shutdown_server()

False

In [ ]:
import asyncio

while True:
    await asyncio.sleep(1)
    running, pending = vllm_server_metrics()
    for _ in range(running - pending + 10):
        ...

In [10]:
def read_last_n_lines(filename: str, n: int) -> str:
    """Read the last n lines of a file efficiently.
    
    Args:
        filename: Path to the file to read
        n: Number of lines to read from end
        
    Returns:
        String containing the last n lines
    """
    from collections import deque
    
    # Use deque with maxlen to efficiently store only n lines
    lines = deque(maxlen=n)
    
    # Read file in chunks from end
    with open(filename, 'rb') as f:
        # Seek to end of file
        f.seek(0, 2)
        file_size = f.tell()
        
        # Start from end, read in 8KB chunks
        chunk_size = 8192
        position = file_size
        
        # Read chunks until we have n lines or reach start
        while position > 0 and len(lines) < n:
            # Move back one chunk
            chunk_size = min(chunk_size, position)
            position -= chunk_size
            f.seek(position)
            chunk = f.read(chunk_size).decode()
            
            # Split into lines and add to deque
            lines.extendleft(chunk.splitlines())
            
        # If we're not at file start, we may have a partial first line
        if position > 0:
            # Read one more chunk to get complete first line
            position -= 1
            f.seek(position)
            chunk = f.read(chunk_size).decode()
            lines[0] = chunk[chunk.rindex('\n')+1:] + lines[0]
            
    return '\n'.join(lines)

print(read_last_n_lines('./logs/vllm.log', 5))

INFO 11-02 15:23:13 metrics.py:345] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%.INFO 11-02 15:23:13 metrics.py:345] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%.
INFO 11-02 15:23:03 metrics.py:345] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%.
INFO 11-02 15:22:53 metrics.py:345] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%.
INFO 11-02 15:22:43 metrics.py:345] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Sw

In [11]:
read_last_n_lines('./logs/vllm.log', 5)

44.5 μs ± 3.2 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [15]:
def parse_last_req_stats(log_str: str) -> tuple[int, int] | None:
    """Parse the last request statistics line from a log string.
    
    Args:
        log_str: String containing log contents
        
    Returns:
        Tuple of (running_reqs, pending_reqs)
    """
    import re
    pattern = r"Running: (\d+) reqs, Swapped: \d+ reqs, Pending: (\d+) reqs"
    matches = list(re.finditer(pattern, log_str))
    if not matches:
        return None
    last_match = matches[-1]
    return (int(last_match.group(1)), int(last_match.group(2)))

parse_last_req_stats(read_last_n_lines('./logs/vllm.log', 5))

(0, 0)

In [17]:
%%timeit
parse_last_req_stats(read_last_n_lines('./logs/vllm.log', 20))

66.4 μs ± 8.19 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
